In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [7]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [8]:
def scrape_links(page_number):
    url=f"https://jakarta.suara.com/indeks?page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "text-list-item-y"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [9]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 20 links from page 1
Total Links: 20


In [10]:
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1')
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "writer"})
        if author_elem:
            author_text = author_elem.find('span')
            author_text = author_text.text
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "date-article"})
        if date_elem:
            date_text = date_elem.find('span')
            date_text = date_text.text
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('div', {"class": "top-detail"})
        if category_elements:
            category_text = category_elements.find('a')
            category_text= category_text.text
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('article', {"class": "detail-content detail-berita"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'region' : 'jakarta',
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [11]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [12]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

,title,author,category,date,content,region,link
0,"Dipimpin Kaesang Pangarep, Begini Harapan PSI ...",Rizki Nurmansyah,\n News ...,"Selasa, 26 September 2023 | 21:15 WIB",SuaraJakarta.id - PSI DKI Jakarta berharap Kae...,jakarta,https://jakarta.suara.com/read/2023/09/26/2115...
1,"4 Fakta Menarik Muhammad Sejahtera Dwi Putra, ...",Husna Rahmayunita,\n News ...,"Selasa, 26 September 2023 | 12:24 WIB",SuaraJakarta.id - Atlet menembak asal Indonesi...,jakarta,https://jakarta.suara.com/read/2023/09/26/1224...
2,"Muhammad Sejahtera Dwi Putra, Atlet Asal Jakar...",Rauhanda Riyantama,\n News ...,"Selasa, 26 September 2023 | 17:22 WIB",SuaraJakarta.id - Kontingen Indonesia kembali ...,jakarta,https://jakarta.suara.com/read/2023/09/26/1722...
3,Kemarau di Jakarta Saat Ini Beda Dibanding 3 T...,Rizki Nurmansyah,\n News ...,"Senin, 25 September 2023 | 19:54 WIB",SuaraJakarta.id - Kepala BMKG Dwikorita Karnaw...,jakarta,https://jakarta.suara.com/read/2023/09/25/1954...
4,Kesulitan Identifikasi 2 Talent Wanita Film De...,Rizki Nurmansyah,\n News ...,"Senin, 25 September 2023 | 21:00 WIB",SuaraJakarta.id - Ditreskrimsus Polda Metro Ja...,jakarta,https://jakarta.suara.com/read/2023/09/25/2100...
5,"Kediri Dholo KOM Challenge 2023, Ratusan Cycli...",Fabiola Febrinastri | Restu Fadilah,\n News ...,"Senin, 25 September 2023 | 11:08 WIB","SuaraJakarta.id - Event bersepeda, Kediri Dhol...",jakarta,https://jakarta.suara.com/read/2023/09/25/1108...
6,"Sambut Tim Verifikator KKS Nasional, Mas Dhito...",Fabiola Febrinastri | Restu Fadilah,\n News ...,"Senin, 25 September 2023 | 10:54 WIB",SuaraJakarta.id - Kabupaten Kediri menjadi sal...,jakarta,https://jakarta.suara.com/read/2023/09/25/1054...
7,"Unggul dalam Transformasi Bisnis, Pos Indonesi...",Fabiola Febrinastri | Restu Fadilah,\n News ...,"Minggu, 24 September 2023 | 14:53 WIB",SuaraJakarta.id - PT Pos Indonesia (Persero) t...,jakarta,https://jakarta.suara.com/read/2023/09/24/1453...
8,Siswa SDN 06 Pesanggrahan Tewas Jatuh dari Lan...,Rizki Nurmansyah,\n News ...,"Selasa, 26 September 2023 | 21:03 WIB","SuaraJakarta.id - Polsek Pesanggrahan, Jakarta...",jakarta,https://jakarta.suara.com/read/2023/09/26/2103...
9,"Hadapi Bali United, Thomas Doll Peringatkan Pe...",Husna Rahmayunita,\n News ...,"Minggu, 24 September 2023 | 10:33 WIB",SuaraJakarta.id - Pelatih Persija Jakarta Thom...,jakarta,https://jakarta.suara.com/read/2023/09/24/1033...


In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/jakartasuara_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')